# Introduction to Natural Language Processing: A Journey through Next Word Prediction

Welcome to the fascinating realm of Natural Language Processing (NLP), a field where language, technology, and human cognition intersect. Here, we embark on a journey to explore one of the most intriguing aspects of NLP: next word prediction. This objective is not just a cornerstone in understanding NLP, but also a window into the intricate ways machines can learn to interpret and generate human language.

Every day, we interact with technology that anticipates what we’re going to say next - be it while typing an email, texting, or searching online. This seemingly magical ability of machines to predict the next word in a sentence is a product of NLP. But how does it work? The answer lies in the ability of algorithms to learn from vast amounts of text data and understand patterns in language use.

Next word prediction is more than a tool for faster typing. It's a fundamental technology in a variety of applications - from enhancing accessibility in communication aids to powering chatbots and virtual assistants. Its implications span various domains, including accessibility, automation, and artificial intelligence, making it a crucial subject in the study of NLP.

At the heart of next word prediction lies the challenge of dealing with the complexity and variability of language. Here, we introduce a probabilistic framework to think about next word prediction.


## Quick Probability Review
Probability is a mathematical framework for quantifying uncertainty. It is a way of thinking about the likelihood of events. In the context of NLP, we can use probability to think about the likelihood of a word occurring in a sentence. For example, the probability of the word "cat" occurring in the sentence "the cat is on the mat" is 1/5, or 0.2.

## Conditional Probability
Conditional probability is the probability of an event occurring given that another event has already occurred. For example, the probability of the word "cat" occurring immediately after the word "the" is 1/2, or 0.5. 

$P(A \mid B)=\frac{P(A \cap B)}{P(B)}$

We can write this as P(cat|the) = 0.5.

Why? Because there are two words that can follow "the" in this sentence: "cat" and "mat". Since there is only one "cat" in the sentence, the probability of "cat" occurring after "the" is 1/2.

## Joint Probability
Joint probability is the probability of two events occurring together. For example, the probability of the word "cat" occurring in the sentence "the cat is on the mat" is 1/5, or 0.2. The probability of the word "mat" occurring in the same sentence is 1/5, or 0.2. The joint probability of "cat" and "mat" occurring together in the sentence is 1/25, or 0.04.

$\mathbb{P}(A \cap B)=\mathbb{P}(B \mid A) \mathbb{P}(A)=\frac{1}{5} \cdot \frac{1}{5}=\frac{1}{25}$

## Chain Rule of Probability
The chain rule of probability is a formula for calculating the joint probability of multiple events. For example, the probability of the words "cat" and "mat" occurring together in the sentence "the cat is on the mat" is 1/25, or 0.04. The probability of the word "the" occurring in the same sentence is 2/5, or 0.4. The chain rule of probability states that the joint probability of "cat", "mat", and "the" occurring together in the sentence is equal to the product of the conditional probabilities of each word given the words that came before it. In this case, the joint probability is equal to 0.04 * 0.4 = 0.016.

$\begin{aligned}
P\left(X_1 \ldots X_n\right) & =P\left(X_1\right) P\left(X_2 \mid X_1\right) P\left(X_3 \mid X_{1: 2}\right) \ldots P\left(X_n \mid X_{1: n-1}\right) \\
& =\prod_{k=1}^n P\left(X_k \mid X_{1: k-1}\right)
\end{aligned}$


## Building a Next Word Predictor

<b>Let's start with a simple probability exercise. Consider the corpus shown below. It consists of 3 sentences contained in a list. Based purely on counting, write a function to compute the probability of any given word in the corpus. This should be a global probability, i.e. the probability of a word occurring anywhere in the corpus.</b>

Note: these solutions have been run through again to minimize loops, as previous solutions had too much nesting. Normally I may prefer to use Numpy vectorization, but Numpy was not a given. As such, it became a challenge of limiting loop nesting only using base Python functions. This source was used due to its clear list of built-in, list, string, and dictionary functions:

https://www.w3schools.com/python/python_ref_functions.asp

In [1]:
corpus = ['this is sentence one',
          'this is sentence two',
          'this is sentence three']

#prob_sentences: returns the probability based on each sentence
#Input: the sentence, the word being compared
#Output: the probability in this sentence
def prob_sentences(sentence, word):
    prob_word = 1/len(corpus) #Get 1 over the number of sentences in the corpus, as this is one sentence out of the corpus
    split_sentence = sentence.split() #Split the sentence into words
    prob_in_sentence = split_sentence.count(word) #Count the number of occurances of the word
    return (prob_in_sentence*prob_word)/len(split_sentence) #Return the occurances in the sentence times the probability of the sentence over the number of words in this sentence, or p(b|a)p(a)

#prob: get the probability of the word for the corpus and return it
#Input: the word, the corpus
#Output: the probability
def prob(word, corpus):
    # your code here
    probs = list(map(prob_sentences, corpus, [word]*len(corpus))) #Get the pribabilities by mapping the prob_sentences function over the corpus, thus returning the result for each sentence
    return sum(probs) #Return the sum of the list of probabilites derived from each sentence to come to a single, final result.
    
print(prob("this", corpus))
print(prob("one", corpus))

0.25
0.08333333333333333


In [2]:
# Test the function - Do not change below this line
def test_prob():
    assert prob('this', corpus) == 0.25
    assert prob('is', corpus) == 0.25
    assert prob('sentence', corpus) == 0.25
    assert round(prob('one', corpus),2) == 0.08
    assert round(prob('two', corpus), 2) == 0.08
    assert round(prob('three', corpus), 2) == 0.08
    print("Your solution is working. Good job!")
    
test_prob()

Your solution is working. Good job!


Now, one of the most important concepts in NLP is that of conditional probability. Conditional probability is the probability of an event occurring given that another event has already occurred. In the context of NLP, we can think of conditional probability as the probability of a word occurring given that another word has already occurred. For example, what is the probability of the word "the" occurring given that the word "quick" has already occurred? We can write this as P(the|quick).

<b> Start by writing a function to return all bigrams in the corpus. The function should take a corpus as input and return a list of bigrams. A bigram is a tuple of two words. For example, the sentence "The quick brown fox jumps over the lazy dog" contains 8 bigrams or co-occuring words: (The, quick), (quick, brown), (brown, fox), (fox, jumps), (jumps, over), (over, the), (the, lazy), (lazy, dog).

We will also add start and end tokens to each sentence in the corpus. This is a common practice in NLP and is useful for computing probabilities. The start token will be represented by the string "s" and the end token by "/s". </b>


In [3]:
# DO NOT CHANGE THIS CORPUS #################
corpus = ['<s> this is sentence one </s>',
          '<s> this is sentence two </s>',
          '<s> this is sentence three </s>']
############################################

#sentence_bigrams: get the bigrams given a sentence
#Input: the sentence
#Output: the bigrams
def sentence_bigrams(sentence):
    bigrams = [] #Create a list for the bigrams
    split_sentence = sentence.split() #Split the sentence so that it gives words instead of characters
    length_sentence = len(split_sentence) #Get the length of the sentence
    
    #For each index 1 to the end, create a bigram tuple out of the previous word and the current word
    for i in range(1, length_sentence):
        bigrams.append((split_sentence[i-1], split_sentence[i])) #Create the bigrams
    
    return bigrams #Return the bigrams

def find_bigrams(corpus) -> list:
    # your code here
    bigram_dict = {} #A dictionary to hold bigrams, specifically to count their commonality later
    bigrams = [] #A list to hold the bigrams
    
    bigram_list = list(map(sentence_bigrams, corpus)) #Map the sentence_bigrams functions over the corpus to get a list of lists of all bigrams from each sentence
    
    #This structure may be a bit odd, but it is to account for base python not having a reduce function
    
    #For each bigram list in the bigram list list, extend a bigram array so these lists come together as a single longer list
    for bigram in bigram_list:
        bigrams.extend(bigram) #Extend the bigram list to make the bigram lists into one list
    
    #Ignore this section. I initially interpreted the assignment to mean all bigrams of a specific count rather
    #    than all bigrams total. I am keeping this part though, as the dictionary assignment may be useful to me later.
    
    #-------------------------------------------------------------------------------------------
    #For each bigram tuple in the bigram list, create a count for it using the bigram dictionary
    #for bigram_tuple in bigrams:
        
        #If the bigram is already in the dictionary, increase the count
        #if bigram_tuple in bigram_dict:
            #bigram_dict[bigram_tuple] += 1 #Increase the count for this bigram
            
        #If the bigram is not already in the list, add it to the list at a count of 1
        #else:
            #bigram_dict[bigram_tuple] = 1 #Add the bigram to the list
    #----------------------------------------------------------------------------------------------
            
    return bigrams #Return the bigrams

print(find_bigrams(corpus))

[('<s>', 'this'), ('this', 'is'), ('is', 'sentence'), ('sentence', 'one'), ('one', '</s>'), ('<s>', 'this'), ('this', 'is'), ('is', 'sentence'), ('sentence', 'two'), ('two', '</s>'), ('<s>', 'this'), ('this', 'is'), ('is', 'sentence'), ('sentence', 'three'), ('three', '</s>')]


In [4]:
# Test the function - Do not change below this line
def test_bigrams():
    assert find_bigrams(corpus)[0] == ('<s>', 'this')
    assert find_bigrams(corpus)[1] == ('this', 'is')
    assert find_bigrams(corpus)[2] == ('is', 'sentence')
    print("Your solution is working. Good job!")
    
test_bigrams()

Your solution is working. Good job!


Now that we have all of the bigrams, we can compute the probability of any word occurring after another as follows:

$P(\text{word2}|\text{word1}) = \frac{Count(\text{word1}, \text{word2})}{Count(\text{word1})}$, where $Count(\text{word1}, \text{word2})$ is the number of times $\text{word1}$ and $\text{word2}$ occur together and $Count(\text{word1})$ is the number of times $\text{word1}$ occurs.

For a clearer idea of why this formula works, refer page 4, chapter 3 of Jurafsky and Martin - https://web.stanford.edu/~jurafsky/slp3/3.pdf

<b>Write a function to compute the conditional probability of a word given another word. The function should take a corpus and two words as input and return the conditional probability of the second word given the first word. For example, the probability of the word "the" occurring given that the word "quick" has already occurred is P(the|quick).</b>

<i>Hint: You can use the function you wrote earlier to get all bigrams in the corpus. You can also use the function you wrote earlier to compute the probability of any given word in the corpus.</i>


In [5]:
bigrams = find_bigrams(corpus)

def conditional_prob(word:str, prev_word:str, bigrams:list) -> float:
    # your code here
    num_prev_word = 0 #Holder for the number of bigrams with the previous word
    num_word_and_prev = 0 #Holder for the number of bigrams with both the previous word and the current word
    
    
    #For each bigram, see if it is prev_word. If so, open the gate. If the gate is open and the second word is word, that means it is the prev_word word bigram
    for bigram in bigrams:
        gate = 0 #Close the gate, the gate being used to eliminate nested if statements
        
        #If the first word is prev_word, add it to the number of prev_word tuples and open the gate
        if bigram[0] == prev_word:
            num_prev_word += 1 #Add this tuple to the number of prev_word tuples
            gate = 1 #Open the gate
            
        #If the gate is open (if the first word of the tuple is prev_word) and the second word of the tuple is word, add that to the counter for the prev_word word bigrams
        if gate == 1 and bigram[1] == word:
            num_word_and_prev += 1 #Up the counter for the number of prev_word word bigrams
    
    return num_word_and_prev/num_prev_word #Return the conditional probability

conditional_prob('one', 'sentence', bigrams)

0.3333333333333333

In [6]:
#Change the cell to a try except cell to still show the divide by 0 while still letting me clear all outputs and rerun the cells
try:
    conditional_prob('one', 'this', corpus)
except:
    print("This version of the code got a divide by 0 error!")

This version of the code got a divide by 0 error!


This is looking good but you might be getting a division by zero error! If this is the case, let's use laplace smoothing to improve it. Laplace smoothing is a technique for dealing with words that do not occur in the corpus. It is a way of adjusting the probability of a word occurring given another word by adding 1 to the numerator and the number of unique words in the corpus to the denominator. This ensures that the probability of a word occurring is never 0. For a clearer idea of why this formula works, refer page 6, chapter 3 of Jurafsky and Martin - https://web.stanford.edu/~jurafsky/slp3/3.pdf

To implement laplace smoothing, we need to modify our conditional probability function as follows:

$P\left(\text{word2}|\text{word1} \right) = \frac{Count(\text{word1},\text{word2})+1}{Count(\text{word1})+V}$ 

where $Count(\text{word1},\text{word2})$ is the number of times $\text{word1}$ and $\text{word2}$ occur together, $Count(\text{word1})$, is the number of times $\text{word1}$ occurs, and $V$ is the number of unique words in the corpus.

In [7]:
bigrams = find_bigrams(corpus)

def conditional_prob(word:str, prev_word:str, bigrams:list, k = 1) -> float:
    # k is the smoothing constant and is set to 1 by default for laplace smoothing
    # your code here
    num_prev_word = 0 #Holder for the number of bigrams with the previous word
    num_word_and_prev = 0 #Holder for the number of bigrams with both the previous word and the current word
    
    #Multiple ideas were tried for the unique section, but this was the fastest. list(set(bigrams, ())) calculated perplexity
    #  176 seconds, while [word for sublist in bigrams for word in sublist] took 24 seconds and this took 0.05 seconds
    
    unique = list(dict(bigrams).keys()) #Convert the bigrams into dictionary, then take a list of their keys
    unique.append("</s>") #Append the end tag, as that is the one item that is in the values but not the keys
    
    #For each bigram, see if it is prev_word. If so, open the gate. If the gate is open and the second word is word, that means it is the prev_word word bigram
    for bigram in bigrams:
        gate = 0 #Close the gate, the gate being used to eliminate nested if statements
        
        #If the first word is prev_word, add it to the number of prev_word tuples and open the gate
        if bigram[0] == prev_word:
            num_prev_word += 1 #Add this tuple to the number of prev_word tuples
            gate = 1 #Open the gate
            
        #If the gate is open (if the first word of the tuple is prev_word) and the second word of the tuple is word, add that to the counter for the prev_word word bigrams
        if gate == 1 and bigram[1] == word:
            num_word_and_prev += 1 #Up the counter for the number of prev_word word bigrams
    
    
    return (num_word_and_prev+1)/(num_prev_word+len(unique)) #Return the conditional probability

conditional_prob('sentence', 'this', bigrams)

0.09090909090909091

In [8]:
conditional_prob('one', 'sentence', bigrams)

0.18181818181818182

In [9]:
# Test the function - Do not change below this line
def test_conditional_prob():
    assert round(conditional_prob('is', 'this', bigrams), 2) == 0.36
    assert round(conditional_prob('sentence', 'this', bigrams), 2) == 0.09
    assert round(conditional_prob('one', 'sentence', bigrams),2) == 0.18
    assert round(conditional_prob('two', 'sentence', bigrams), 2) == 0.18
    assert round(conditional_prob('three', 'sentence', bigrams), 2) == 0.18
    print("Your solution is working. Good job!")

test_conditional_prob()

Your solution is working. Good job!


Now, let's try and predict the next word in a sentence. This is essentially what an advanced chatbot like ChatGPT also tries to do, although ChatGPT uses a much bigger corpus and a more sophisticated algorithm. We will use the same corpus as before, but this time we will use the conditional probability formula to predict the next word in a sentence. We will also use the start and end tokens to help us compute the probabilities.

<b>Write a function to predict the next word in a sentence. The function should take a corpus and a sentence as input and return the most likely next word in the sentence. For example, if the sentence is "the quick brown", the function should return "fox".</b>

<i>Hint: You can use the conditional probability function you wrote earlier to compute the probability of each word in the vocabulary occurring after the last word in the sentence. The word with the highest probability is the most likely next word. In other words, check the probability of each possible word in the corpus with respect to the given previous word and pick the one with the maximum probability. To make your algorithm more efficient, you can ignore non-co-occuring pairs when you build the vocabulary. </i>

In [10]:
def predict_next_word(prev_word: str, bigrams: list) -> str:
    # your code here
    prob_dict = {} #Create a dictionary to put the possible words with their probabilities
    highest_prob = 0 #Set the highest probability found so far for comparison
    highest_prob_word = "" #Set the highest probability word thus far to go with the highest probability
    
    #For each bigram, find the possible words based on the prev_word and their probability, then put them in a dictionary
    for bigram in bigrams:
        
        #If the first word in the pair is the prev_word, get the probability of the second word being correct and put it in the dictionary
        if bigram[0] == prev_word:
            prob_dict[bigram[1]] = conditional_prob(bigram[1], prev_word, bigrams) #Put the word and probability in the dictionary
    
    #For each word probability pair in the dictionary, see if it has the highest probability. If so, set it as such
    for word, prob in prob_dict.items():
        
        #See if this probability is higher than the current highest probability
        if prob > highest_prob:
            highest_prob = prob #Set the new highest probability
            highest_prob_word = word #Set the new highest probability word
            
    return highest_prob_word #Return the word with the highest probability

predict_next_word('this', bigrams)

'is'

In [11]:
predict_next_word('sentence', bigrams)

'one'

In [12]:
# Test the function - Do not change below this line
def test_predict_next_word():
    assert predict_next_word('this', bigrams) == 'is'
    assert predict_next_word('is', bigrams) == 'sentence'
    print("Your solution is working. Good job!")
    
test_predict_next_word()

Your solution is working. Good job!


Let's try and predict an entire sentence now.

<b>Write a function that takes an initial string and a word limit as input and returns a sentence of the specified length. The initial string is the starting point for the sentence. For example, if the initial string is "the quick brown" and the word limit is 5, the function should return "the quick brown fox jumps over". The function should use the predict_next_word function you wrote earlier to predict the next word in the sentence.</b>

<i>Hint: You can use a for loop to predict the next word in the sentence. The initial string is the starting point for the sentence. At each iteration, you can add the predicted word to the sentence and use the predicted word as the new initial string. You can stop the loop when the word limit is reached. </i>

<i>Hint: You can use try, except to handle the case where the initial string is not in the corpus. If you don't catch the exception, you might encounter a value error. </i>

In [13]:
def predict_sentence(input_sentence, bigrams, limit=2):
    # your code here
    count = 0 #Create a count variable so that the limit is not breached
    sentence = input_sentence #Create a sentence variable to hold the sentence as it changes
    
    #While not at the limit, generate next words. A while loop is used instead of a for loop to allow for more intuitive breaking
    while count < limit:
        split_sentence = sentence.split() #Split the sentence so that indexing gives a whole word rather than a letter
        next_word = predict_next_word(split_sentence[-1], bigrams) #Predict the next word based on the previous word
        sentence = f"{sentence} {next_word}" #Add the new word to the sentence, using an fstring so that all of the +s and extra spaces are not required
        count += 1 #Increase the counter
        
        #If the next word is the ending, break from the loop
        if next_word == "</s>":
            break #Break from the loop
        
    return sentence #Return the generated sentence

In [14]:
predict_sentence("This is", bigrams, limit = 7)

'This is sentence one </s>'

In [15]:
# Test the function - Do not change below this line
def test_predict_sentence():
    assert predict_sentence("This is", bigrams, limit = 7) == 'This is sentence one </s>'
    assert predict_sentence("This is", bigrams, limit = 1) == 'This is sentence'
    print("Your solution is working. Good job!")

test_predict_sentence()

Your solution is working. Good job!


Now let's try this with a real corpus. We will use the Brown corpus, which is a collection of 500 samples of English-language text, totaling roughly one million words. The Brown corpus is a good choice for this exercise because it is a balanced corpus, meaning it contains text from a variety of genres. This makes it a good representation of the English language. The Brown corpus is also a tagged corpus, meaning it contains information about the part of speech of each word. We will use this information later in the course.

In [16]:
import nltk

# import the brown corpus from nltk
from nltk.corpus import brown

import re

# download the corpus
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\lunam\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [17]:
# get the corpus
corpus_brown = brown.sents()

corpus_brown = [' '.join(sentence) for sentence in corpus_brown]
# Add start and end tags
corpus_brown = ['<s> ' + sentence + ' </s>' for sentence in corpus_brown][:3000]
print(corpus_brown[0:2])
bigrams_brown = find_bigrams(corpus_brown)

["<s> The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place . </s>", "<s> The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . </s>"]


Now, let's try and predict a series of next words using the Brown corpus. The operation below might take a few seconds to run based on your hardware.

In [18]:
import time #Allowing this import to show how slow my system can be and, thus, why I originally wanted to go for vectorization

start = time.time()
print(f"Sentence: {predict_sentence('This', bigrams_brown, limit = 20)}")
end = time.time()

print(f"Time taken: {(end - start)/60} minutes")

Sentence: This is a year . </s>
Time taken: 2.27727267742157 minutes


This version with the list(dict.keys) in the conditional probability may show this time, but other ideas that were tried took several hours.

In [19]:
for i in range(5):
    print(f"Sentence: {predict_sentence('<s>', bigrams_brown, limit = 20)}")

Sentence: <s> The President Kennedy administration . </s>
Sentence: <s> The President Kennedy administration . </s>
Sentence: <s> The President Kennedy administration . </s>
Sentence: <s> The President Kennedy administration . </s>
Sentence: <s> The President Kennedy administration . </s>


## A primer on perplexity
Perplexity is a measurement in Natural Language Processing (NLP) used to evaluate language models. It's a measure of how well a probability model predicts a sample. A lower perplexity score indicates better performance of the model.

The perplexity of a sentence is calculated as the inverse probability of the sentence, normalized by the number of words. In other words, it's the geometric mean of the inverse conditional probability of each word given the previous word in the sentence.

$\text{Perplexity}(W)=\sqrt[N]{\frac{1}{P(w_1, w_2, \ldots, w_N)}}=\sqrt[N]{\prod_{i=1}^N \frac{1}{P(w_i \mid w_1, \ldots, w_{i-1})}}$

where $N$ is the number of words in the sentence and $P(w_i \mid w_1, \ldots, w_{i-1})$ is the conditional probability of the i'th word given the previous words in the sentence.

In our case, since we only have bigrams, the formula becomes:
$\text{Perplexity}(W)=\sqrt[N]{\frac{1}{P(w_1, w_2, \ldots, w_N)}}=\sqrt[N]{\prod_{i=1}^N \frac{1}{P(w_i \mid w_{i-1})}}$

Perplexity is a useful metric for evaluating language models because it is a measure of how well a probability model predicts a sample. A lower perplexity score indicates better performance of the model.

Let's try and calculate the perplexity for a few sentences in the Brown corpus. We will use the same corpus as before, but this time we will use the perplexity formula to calculate the perplexity of each sentence. We will also use the start and end tokens to help us compute the perplexity.

In [20]:
def perplexity(sentence, bigrams) -> float:
    # your code here
    perplexity_denom = 1 #A holder variable for the probability, starting at 1 as 1*x = x
    
    new_sentence = sentence #Set the sentence. This is a remnant from when adding the tags was the expectation and I do not want to change everything to accommodate.
    
    split_sentence = new_sentence.split() #Split the sentence for better counting
    len_sentence = len(split_sentence) #Get the number of words
    prev_word = "" #Set a previous word variable so (nothing, <s>) is not included
    
    #For each word, get its probability with the previous word. If there is no previous word, set this word to the previous word
    for word in split_sentence:
        #If there is a previous word, compute the probability
        if prev_word != "":
            perplexity_denom *= conditional_prob(word, prev_word, bigrams) #Multiply the probability by the base probability
        prev_word = word #Set the current word to the previous word
        
    return (1/perplexity_denom)**(1/len_sentence) #Compute the perplexity (1/probability)**(1/number of words) and return it
    
perplexity("This is", bigrams)

2.8284271247461903

In [21]:
# Test the function - Do not change below this line
def test_perplexity():
    corpus = ['<s> this is sentence one </s>',
              '<s> this is sentence two </s>',
              '<s> this is sentence three </s>']
    bigrams = find_bigrams(corpus)
    perplexity_score = perplexity("This is ", bigrams)
    assert round(perplexity_score, 2) == 2.83
    print("Your solution is working. Good job!")
    
test_perplexity()

Your solution is working. Good job!


Now pick 5 sentences from the Brown corpus and calculate the perplexity of each sentence. The code to import and load the corpus is provided above. Also calculate the average perplexity of the subset of the brown corpus chosen above.

You can also make up your own sentences and calculate the perplexity of each sentence. How does the perplexity of your sentences compare to the perplexity of the sentences in the Brown corpus?

In [22]:
# Your code here

#Timing taken for optimization purposes. The perplexity only uses conditional probability, so this was best to make changes.
start = time.time()
perplexity("This is ", bigrams_brown)
end = time.time()

print(f"Time taken: {end-start} Seconds")

Time taken: 0.06854438781738281 Seconds


In [23]:
#Pseudo-random number generation ideas discussed here since I could not add random.random
# https://stackoverflow.com/questions/11946622/implementation-of-random-number-generator
brown_corpus_length = len(corpus_brown) #Get the length of the corpus
prev_factor = 42 #Choose a prev factor
change_factor = 24 #Choose a change factor for the equation

#Pseudo-randomly choose 5 sentences and find their perplexity
for i in range(1, 6):
    change_factor = (change_factor**i * prev_factor**i + len(corpus_brown[i]))%brown_corpus_length #Pseudo-randomly choose a sentence from the corpus
        
    prev_factor = change_factor #Update the prev factor
    change_factor = change_factor - i #Update the change factor
    sentence = corpus_brown[change_factor] #Pull the selected sentence
    
    #Print information about the random selection
    print(f"Sentence Number: {change_factor}")
    print(f"Sentence: {sentence}")
    print(f"Perplexity: {perplexity(sentence, bigrams_brown)}")
    print("---------------------------------------------------")

Sentence Number: 1255
Sentence: <s> It was Gardner's second run batted in of the game and his only ones of the year . </s>
Perplexity: 666.7970999406034
---------------------------------------------------
Sentence Number: 630
Sentence: <s> The Democratic leadership , however , hopes to pass it sometime this week . </s>
Perplexity: 748.7320729621073
---------------------------------------------------
Sentence Number: 191
Sentence: <s> These , he said , are `` two of the principal underlying causes for family breakups leading to ADC '' . </s>
Perplexity: 928.2808278556488
---------------------------------------------------
Sentence Number: 2195
Sentence: <s> He indicated that requests would be made for more U.S. arms and more U.S. military advisers . </s>
Perplexity: 1237.8332490003231
---------------------------------------------------
Sentence Number: 1263
Sentence: <s> -- Boston Red Sox Outfielder Jackie Jensen said Monday night he was through playing baseball . </s>
Perplexity: 1402.

In [24]:
print(perplexity("short sentence", bigrams_brown))
print(perplexity("""This is a long sentence: 
According to all known laws of aviation, 
there is no way a bee should be able to fly. 
Its wings are too small to get its fat little body off the ground.
The bee, of course, flies anyway because bees don't care what humans think is impossible.""", bigrams_brown))

103.60501918343532
5454.223720118167


The full Bee Movie script (found here: https://courses.cs.washington.edu/courses/cse163/20wi/files/lectures/L04/bee-movie.txt) caused a divide by 0 error on the perplexity, but the intro does just fine. It is interesting, though, that the Bee Movie script intro only has a perplexity of 5282 when some of the ones from the original corpus near 2000. 

Write a brief half-page report on the concepts you explored in this exercise. You can use the questions below as a guide.
1) What is the conditional probability of a word occurring given its previous word and how does this probability change when you use laplace smoothing?
2) What do you expect will happen when you use trigrams instead of bigrams to predict the next word in a sentence? Does the perplexity increase or decrease? Why?
3) What do you expect will happen when you use a larger corpus to predict the next word in a sentence?

That's all folks! You've built yourself a next word predictor. Now go put it out there and get acquired by Microsoft for a few billion dollars.

Further exercises (will not be graded):

- Try to improve the model by using trigrams instead of bigrams. A trigram is a tuple of three words. For example, the sentence "The quick brown fox jumps over the lazy dog" contains 7 trigrams: (The, quick, brown), (quick, brown, fox), (brown, fox, jumps), (fox, jumps, over), (jumps, over, the), (over, the, lazy), (the, lazy, dog). You can use the same formula to compute the conditional probability of a word given two other words. For example, the probability of the word "the" occurring given that the words "quick" and "brown" have already occurred is P(the|quick, brown). You can also use the start and end tokens to help you compute the probabilities. For example, the probability of the word "the" occurring given that the words "s" and "quick" have already occurred is P(the|s, quick). You can use the same function you wrote earlier to compute the conditional probability of a word given two other words. The function should take a corpus and three words as input and return the conditional probability of the third word given the first two words. For example, the probability of the word "the" occurring given that the words "quick" and "brown" have already occurred is P(the|quick, brown).